In [ ]:
!pip install efficientnet_pytorch
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
import matplotlib.pyplot as plt
from PIL import Image # pil及cv库，用于图片的打开修改等工作
import torchvision
import cv2
print(torch.__version__)  # 检查pytorch的版本
print(torch.cuda.is_available())  # 确定你的电脑的cuda版本，需要与你安装的pytorch的cuda版本匹配
print(torch.version.cuda)  # 确定你的电脑的cuda版本，需要与安装的pytorchd版本匹配
PATH = "../input/sorghum-id-fgvc-9/"
TRAIN_IMG = PATH + "train_images/"
TEST_IMG = PATH+"test/"
IMG_WIDE = 224
IMG_HEIGHT =224
NUM_CLASS = 100

class opts:
    batch_size = 10
    lr = 0.0008
    weight_delay = 1e-4
    start_epoch = 20
    epoch = 25
    use_gpu = True;
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(opts.device)

In [ ]:
#!git clone https://github.com/lukemelas/EfficientNet-PyTorch
#!cd EfficientNet-Pytorch
#!pip install -e .


In [ ]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_name('efficientnet-b4')

model.load_state_dict(torch.load("../input/epoch59/epoch20.pt", map_location='cpu')) #加载之前保存的模型参数

#from efficientnet_pytorch import EfficientNet
#model = EfficientNet.from_pretrained('efficientnet-b4')
#for k,v in model.named_parameters():
#    print('{}: {}'.format(k, v.requires_grad))
i=0
#for k,v in model.named_parameters():
#    i+=1
#    if ('bn' in k) and i<194:
#        v.requires_grad = False
#for k,v in model.named_parameters():
#    print('{}: {}'.format(k, v.requires_grad))
#for k,v in model.named_parameters():
#    print('{}: {}'.format(k, v.requires_grad))




In [ ]:
train_msg = pd.read_csv(PATH + "train_cultivar_mapping.csv")
train_msg.dropna(inplace=True)  #这行可以去掉标题和存在数据缺失的项
print (f"number of train data = {len(train_msg)}")
train_msg.head()
sorghum_type = list(train_msg["cultivar"].unique())
print(len(sorghum_type))

In [ ]:
#这一部分主要是pandas的应用，注意以下这些函数：apply，map，unique，index函数以及python的lambda功能

train_msg["dir"] = train_msg["image"].apply( lambda image:TRAIN_IMG+image )
train_msg["dir_is_true"] = train_msg["dir"].apply(lambda dir: os.path.exists(dir))
train_msg["sorghum_type"] = train_msg["cultivar"].map(lambda type:sorghum_type.index(type))
train_msg.head()

In [ ]:
RATIO = 0.9
tfms = transforms.Compose([transforms.RandomAffine(30,scale=(1,1.2)),
    transforms.RandomResizedCrop(512), 
    transforms.RandomAdjustSharpness(2.),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(90),
    transforms.RandomApply(torch.nn.ModuleList([transforms.GaussianBlur(5)])),
    transforms.ToTensor(),
    transforms.Normalize([0.385, 0.356, 0.306], [0.229, 0.224, 0.225]),])
tsfm = transforms.Compose([transforms.RandomResizedCrop(512), 
    transforms.ToTensor(),
    transforms.Normalize([0.385, 0.356, 0.306], [0.229, 0.224, 0.225]),])
class MyDataset(Dataset):
    def __init__(self,train_msg,mode = "train",trans = tfms):
        """
        :param train_msg: 整理后的地址和品种信息
        :param mode: 训练集/验证集
        :param trans: 数据预处理函数
        """
        self.trans = trans
        self.mode = mode
        num_all = int(len(train_msg))
        num_train = int(num_all * RATIO);
        num_val = num_all - num_train;
        if self.mode == "train":
            img_list = train_msg[:num_train]
        elif self.mode == "val":
            img_list = train_msg[num_train:num_all]
        self.length = len(img_list)
        self.label = img_list["sorghum_type"].values
        self.img_dir =  img_list["dir"].values
        self.img_true = img_list["dir_is_true"].values
    
    def __len__(self):
        return self.length
    def __getitem__(self,item ):
        if self.trans is None:
                trans = transforms.Compose([
                    transforms.Resize([IMG_WIDE, IMG_HEIGHT]),
                    transforms.ToTensor(),
                ])
        else:
            trans = self.trans
            
        if self.img_true[item]:
            label = torch.tensor(self.label[item], dtype=torch.long)
            image_path = self.img_dir[item]
            image = Image.open(image_path)
            #image = cv2.imread(image_path)
            #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = trans(image)
        else:
            self.length -= 1
            print(f"loss picture {item}")
        return image,  label

class TestDataset(Dataset):
    def __init__(self,test_msg,trans = tsfm):
        self.trans = trans
        self.length = len(test_msg)
        self.name  =test_msg["filename"].values
        self.img_dir =  test_msg["dir"].values
        self.img_true = test_msg["dir_is_true"].values
    def __len__(self):
        return self.length
    def __getitem__(self,item ):
        if self.trans is None:
                trans = transforms.Compose([
                    transforms.Resize([IMG_WIDE, IMG_HEIGHT]),
                    transforms.ToTensor(),
                ])
        else:
            trans = self.trans
            
        if self.img_true[item]:
            image_path = self.img_dir[item]
            image = Image.open(image_path)
            image = trans(image)
        else:
            self.length -= 1
            print(f"loss picture {item}")
        return image,self.name[item]
        

In [ ]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
dataset_val = MyDataset(train_msg, mode='train')
val_loader = DataLoader(dataset_val, batch_size=40, shuffle=True)
i = 0
#取验证集中的40张图片显示
for batch in val_loader:
    
    if (i == 0):
        images,labels= batch
        print(type(images))
        i += 1
    else:
        break
grid = torchvision.utils.make_grid(images, nrow=10)
print(labels)
plt.figure(figsize=(20, 20))
plt.imshow(np.transpose(grid, (1, 2, 0)))
plt.show()

In [ ]:
from collections import namedtuple
_GoogLeNetOuputs = namedtuple('GoogLeNetOuputs', ['logits', 'aux_logits2', 'aux_logits1'])
class GoogLeNet(nn.Module):

    def __init__(self, num_classes=100, aux_logits=False, transform_input=False, init_weights=True):
        super(GoogLeNet, self).__init__()
        self.aux_logits = aux_logits
        self.transform_input = transform_input

        self.conv1 = BasicConv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(3, stride=2, ceil_mode=True)   #向上取整
        self.conv2 = BasicConv2d(64, 64, kernel_size=1)
        self.conv3 = BasicConv2d(64, 192, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(3, stride=2, ceil_mode=True)

        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)

        if aux_logits:
            self.aux1 = InceptionAux(512, num_classes)
            self.aux2 = InceptionAux(528, num_classes)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(1024, num_classes)

        if init_weights:
            self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                import scipy.stats as stats
                X = stats.truncnorm(-2, 2, scale=0.01)
                values = torch.as_tensor(X.rvs(m.weight.numel()), dtype=m.weight.dtype)
                values = values.view(m.weight.size())
                with torch.no_grad():
                    m.weight.copy_(values)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        if self.transform_input:
            x_ch0 = torch.unsqueeze(x[:, 0], 1) * (0.229 / 0.5) + (0.485 - 0.5) / 0.5
            x_ch1 = torch.unsqueeze(x[:, 1], 1) * (0.224 / 0.5) + (0.456 - 0.5) / 0.5
            x_ch2 = torch.unsqueeze(x[:, 2], 1) * (0.225 / 0.5) + (0.406 - 0.5) / 0.5
            x = torch.cat((x_ch0, x_ch1, x_ch2), 1)
        # N x 3 x 224 x 224
        x = self.conv1(x)
        # N x 64 x 112 x 112
        x = self.maxpool1(x)
        # N x 64 x 56 x 56
        x = self.conv2(x)
        # N x 64 x 56 x 56
        x = self.conv3(x)
        # N x 192 x 56 x 56
        x = self.maxpool2(x)
        # N x 192 x 28 x 28
        x = self.inception3a(x)
        # N x 256 x 28 x 28
        x = self.inception3b(x)
        # N x 480 x 28 x 28
        x = self.maxpool3(x)
        # N x 480 x 14 x 14
        x = self.inception4a(x)
        # N x 512 x 14 x 14
        if self.training and self.aux_logits:
            aux1 = self.aux1(x)

        x = self.inception4b(x)
        # N x 512 x 14 x 14
        x = self.inception4c(x)
        # N x 512 x 14 x 14
        x = self.inception4d(x)
        # N x 528 x 14 x 14
        if self.training and self.aux_logits:
            aux2 = self.aux2(x)

        x = self.inception4e(x)
        # N x 832 x 14 x 14
        x = self.maxpool4(x)
        # N x 832 x 7 x 7
        x = self.inception5a(x)
        # N x 832 x 7 x 7
        x = self.inception5b(x)
        # N x 1024 x 7 x 7
        x = self.avgpool(x)
        # N x 1024 x 1 x 1
        x = x.view(x.size(0), -1)
        # N x 1024
        x = self.dropout(x)
        x = self.fc(x)
        # N x 100 (num_classes)
        if self.training and self.aux_logits:
            return _GoogLeNetOuputs(x, aux2, aux1)
        return x


class Inception(nn.Module):     #Inception模块

    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
        super(Inception, self).__init__()

        self.branch1 = BasicConv2d(in_channels, ch1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            BasicConv2d(in_channels, ch3x3red, kernel_size=1),
            BasicConv2d(ch3x3red, ch3x3, kernel_size=3, padding=1)
        )

        self.branch3 = nn.Sequential(
            BasicConv2d(in_channels, ch5x5red, kernel_size=1),
            BasicConv2d(ch5x5red, ch5x5, kernel_size=3, padding=1)
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1, ceil_mode=True),
            BasicConv2d(in_channels, pool_proj, kernel_size=1)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)

        outputs = [branch1, branch2, branch3, branch4]
        return torch.cat(outputs, 1)


class InceptionAux(nn.Module):      #辅助分支

    def __init__(self, in_channels, num_classes):
        super(InceptionAux, self).__init__()
        self.conv = BasicConv2d(in_channels, 128, kernel_size=1)

        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        # aux1: N x 512 x 14 x 14, aux2: N x 528 x 14 x 14
        x = F.adaptive_avg_pool2d(x, (4, 4))
        # aux1: N x 512 x 4 x 4, aux2: N x 528 x 4 x 4
        x = self.conv(x)
        # N x 128 x 4 x 4
        x = x.view(x.size(0), -1)
        # N x 2048
        x = F.relu(self.fc1(x), inplace=True)
        # N x 1024
        x = F.dropout(x, 0.7, training=self.training)
        # N x 1024
        x = self.fc2(x)
        # N x num_classes

        return x


class BasicConv2d(nn.Module):       #Conv2d+BN+Relu

    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

def test():
    net = GoogLeNet()
    x = torch.randn(1,3,224,224)
    y = net(x)
    print(y.size())

test()

In [ ]:
#model = GoogLeNet()
optimizer = torch.optim.SGD(model.parameters(), lr=opts.lr, momentum=0.9)  # 建立优化器
criterion2 = nn.CrossEntropyLoss()
if opts.use_gpu:
    model.to(opts.device)

train_dataset = MyDataset(train_msg, mode='train')
val_dataset = MyDataset(train_msg, mode='val')
train_loader = DataLoader(train_dataset, opts.batch_size, shuffle=False, num_workers=0, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=0)
print(" dateset prepare down!")

num_train = len(train_dataset)  # 记录训练集及验证集的带训练图片标签个数
num_val = len(val_dataset)

for epoch in range(opts.start_epoch + 1, opts.epoch + 1):
    model.train()
    losssum = 0.
    
    for i, (imgs, labels) in enumerate(train_loader):
        #print(" have down %d%% image in epoch %d" % (int((i * opts.batch_size / num_train) * 100), epoch),end="\r")
        if(i%100==0):
            print(" have down %d%% image in epoch %d" % (int((i * opts.batch_size / num_train) * 100), epoch))
        #label =  torch.zeros(opts.batch_size, NUM_CLASS )
        #print (labels.size())
        #print (label.size())
        #print (label);
        if opts.use_gpu:  # 若使用GPU，则将数据转移到GPU中
            imgs = imgs.to(opts.device)
            labels = labels.to(opts.device)
        preds = model(imgs)  # 前向传播
        loss = criterion2(preds,labels ) # 计算损失
        optimizer.zero_grad()  # 梯度清零
        loss.backward()  # 反向传播
        optimizer.step()  # 优化网络参数
        losssum += float(loss.item())
    print(" Epoch %d/%d | avg_loss = %.3f" % (epoch, opts.epoch, losssum / num_train))  # 统计本次epoch的数据
    
    model.eval()
    with torch.no_grad():
        num = 0
        for i, (imgs, labels) in enumerate(val_loader):
            if opts.use_gpu:  # 若使用GPU，则将数据转移到GPU中
                imgs = imgs.to(opts.device)
                labels = labels.to(opts.device)
            preds = model(imgs)
            ans = preds.argmax(dim=1)

            if ans == labels:
                num += 1
        print(" Epoch %d/%d | accurate = %.3f" % (epoch, opts.epoch, num / num_val))  # 统计本次epoch的数据
    
    if epoch%5 == 0:
        print(" Save model")
        model_name = "epoch%d.pt" % epoch  # 保存模型命名
        save_dir = os.path.join('./', model_name)  # 存入定好的checkpoints地址中
        torch.save(model.state_dict(), save_dir)  # 保存结果为pt文件。
    

In [ ]:
class test_opts():
    def __init__(self):
        self.PRETRAIN = False  # 是否使用之前完成训练的模型
        self.CHECKPOINTS_DIR = PATH + "\model\epoch49.pt"  # 先前保存的模型的地址和名称
        self.DEVICE = "cpu"

test_opt = test_opts()
#model = GoogLeNet()
if test_opt.PRETRAIN:
    model = YoloNet()  # 建立测试模型
    model.load_state_dict(torch.load(test_opt.CHECKPOINTS_DIR, map_location='cpu')) #加载之前保存的模型参数

In [ ]:
test_msg = pd.read_csv(PATH + "sample_submission.csv")
test_msg.dropna(inplace=True) 
print (f"number of train data = {len(test_msg)}")
test_msg.head()

In [ ]:
test_msg["dir"] = test_msg["filename"].apply( lambda image:TEST_IMG+image )
test_msg["dir_is_true"] = test_msg["dir"].apply(lambda dir: os.path.exists(dir))
test_msg["sorghum_type"] = test_msg["cultivar"].map(lambda type:sorghum_type.index(type))
test_msg.head()

In [ ]:
test_dataset = TestDataset(test_msg)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0, drop_last=False)
num_test = len(test_dataset)
print(num_test)

model.cuda()
model.eval()
predictions = []
for i, (imgs,labels) in enumerate(test_loader):
    imgs=imgs.cuda()
    with torch.no_grad():
        preds = model(imgs).detach().cpu()
        #predictions.append([preds.argmax(dim=1),labels])
        predictions.append(preds.argmax(dim=1))
        if i%100 == 0:
            print(" have down %d%% image" % (int((i / num_test) * 100)),end="\r")
            
#predictions = [[sorghum_type[pred],labels] for pred,labels in predictions]
predictions = [sorghum_type[pred%100] for pred in predictions]
sub = pd.read_csv(PATH + "sample_submission.csv")
sub["cultivar"] = predictions
sub.to_csv('submission.csv', index=False)
sub.head()
    